In [1]:
import pandas as pd
import math as m
import numpy as np
import matplotlib.pyplot as plt
from timeit import default_timer as timer


def  GBM_generator(mu, sigma, T, dt, S0):
    N = round(T / dt)
    t = np.linspace(0, T, N)
    W = np.random.standard_normal(size=N)
    W = np.cumsum(W) * np.sqrt(dt)  ### standard brownian motion ###
    X = (mu - 0.5 * sigma ** 2) * t + sigma * W
    S = S0 * np.exp(X)  ### geometric brownian motion ###

    return t, S

def prod(j, array):
    p = 1
    if j == 0:
        return p * (1 - array[0])
    else:
        return prod(j-1, array) * (1-array[j])


def transform_csv(df):

    # print("Кол-во строк в таблице (старт): \n", len(df), "\n")
    # print("Таблица (оглавление) (старт): \n", df.head(10), "\n")

    df['p_sell'] = 0
    df['p_buy'] = 0
    df['day_profit'] = 0
    df['total_profit'] = 0
    df['sell/buy'] = 0
    df['count_sell'] = 0
    df['count_buy'] = 0
    df['count_total_buy'] = 0
    df['costs_of_bying'] = 0
    df['sum_invested'] = 0

    # print("Таблица (оглавление) (после добавления столбцов): \n", df.head(), "\n")



def model_5k(df):

    column_p_sell = 'p_sell'
    column_p_buy = 'p_buy'
    column_price = 'Price'
    column_sell_buy = 'sell/buy'
    column_day_profit = 'day_profit'
    column_profit = 'total_profit'
    column_count_sell = 'count_sell'
    column_count_buy = 'count_buy'
    column_count_total_buy = 'count_total_buy'
    column_costs_of_bying = 'costs_of_bying'
    column_sum_invested = 'sum_invested'
    column_ticker = 'ticker'

    p0 = df.loc[0, column_price]

    # визначаємо яку кількість акцій потрібно купувати на відповідному етапі докуповування
    number = []
    for j in range(0, len(amounts_S)):
        if j == 0:
            number.append(amounts_S[j] / p0)
        else:
            number.append(amounts_S[j] / (p0 * prod(j, procent)))

    k0 = number[0]
    K = k0
    S0 = k0 * p0
    C = S0
    Profit = 0
    t = 0
    p_sell = p0 * (1 + r / 100)
    p_buy = p0 * (1 - procent[1])

    df.loc[0, column_p_sell] = p_sell
    df.loc[0, column_p_buy] = p_buy
    df.loc[0, column_sell_buy] = 'buy'
    df.loc[0, column_day_profit] = K * df.loc[0, column_price] - C
    df.loc[0, column_profit] = Profit
    df.loc[0, column_count_buy] = k0
    df.loc[0, column_count_total_buy] = K
    df.loc[0, column_costs_of_bying] = S0
    df.loc[0, column_sum_invested] = C

    for i in range(1, len(df)):
        count_days[t] = count_days[t] + 1
        # count_days

        if df[column_price][i] > p_sell:
            if t < len(amounts_S):
                count_step[t] = count_step[t] + 1
                # count_step
                size_profit[t] = size_profit[t] + K * df[column_price][i] - C
                # size_profit
            else:
                pass
            Profit = Profit + (K * df[column_price][i] - C)
            df.loc[i, column_profit] = Profit
            df.loc[i, column_count_sell] = K

            p0 = df[column_price][i]
            p_sell = p0 * (1 + r / 100)
            p_buy = p0 * (1 - procent[1])

            df.loc[i, column_p_sell] = p_sell
            df.loc[i, column_p_buy] = p_buy
            df.loc[i, column_sell_buy] = 'sell'
            df.loc[i, column_day_profit] = K * df.loc[i, column_price] - C
            df.loc[i, column_sum_invested] = C

            number = []
            for j in range(0, len(amounts_S)):
                if j == 0:
                    number.append(amounts_S[j] / p0)
                else:
                    number.append(amounts_S[j] / (p0 * prod(j, procent)))
            k0 = number[0]
            K = k0
            S0 = k0 * p0
            C = S0
            t = 0

        elif df[column_price][i] < p_buy:
            t = t + 1

            if t < len(amounts_S):
                k0 = number[t]
                K = K + k0
                p0 = df[column_price][i]
                S0 = k0 * p0
                C = C + S0

                p_sell = (C / K) * (1 + r_fin / 100)

                if (t + 1) < len(amounts_S):
                    p_buy = p0 * (1 - procent[1])
                else:
                    p_buy = p0 * (1 - procent_loss / 100)

                df.loc[i, column_p_sell] = p_sell
                df.loc[i, column_p_buy] = p_buy
                df.loc[i, column_profit] = Profit
                df.loc[i, column_sell_buy] = 'buy'
                df.loc[i, column_count_buy] = k0
                df.loc[i, column_count_total_buy] = K
                df.loc[i, column_costs_of_bying] = S0
                df.loc[i, column_sum_invested] = C

            elif t == len(amounts_S):
                count_step[t] = count_step[t] + 1
                count_step
                df.loc[i, column_day_profit] = K * df.loc[i, column_price] - C
                size_profit[t] = size_profit[t] + K * df[column_price][i] - C
                df.loc[i, column_sell_buy] = 'StopLoss'
                df.loc[i, column_count_sell] = K
                df.loc[i, column_sum_invested] = C

                Profit = Profit + K * df[column_price][i] - C
                df.loc[i, column_profit] = Profit

                p0 = df[column_price][i]
                p_sell = p0 * (1 + r / 100)
                p_buy = p0 * (1 - procent[1])

                df.loc[i, column_p_sell] = p_sell
                df.loc[i, column_p_buy] = p_buy

                number = []
                for j in range(0, len(amounts_S)):
                    if j == 0:
                        number.append(amounts_S[j] / p0)
                    else:
                        number.append(amounts_S[j] / (p0 * prod(j, procent)))
                #                 print('Stoploss', K * df[column_price][i] - C)
                k0 = number[0]
                K = k0
                C = k0 * p0
                t = 0
        else:
            df.loc[i, column_p_sell] = df.loc[i - 1, column_p_sell]
            df.loc[i, column_p_buy] = df.loc[i - 1, column_p_buy]
            df.loc[i, column_profit] = Profit
            df.loc[i, column_sum_invested] = C
            df.loc[i, column_sell_buy] = 'waiting'

    df['day_profit'] = round(df['day_profit'], 0)
    df['total_profit'] = round(df['total_profit'], 0)
    #     df['Date'] = pd.to_datetime(df['Date'], format="%Y/%m/%d")
    df['p_buy'] = round(df['p_buy'], 2)
    df['p_sell'] = round(df['p_sell'], 2)
    df['count_buy'] = round(df['count_buy'], 2)
    df['count_sell'] = round(df['count_sell'], 2)
    df['count_total_buy'] = round(df['count_total_buy'], 2)
    df['costs_of_bying'] = round(df['costs_of_bying'], 2)
    df['sum_invested'] = round(df['sum_invested'], 2)

    return df, Profit


# START
start = timer()

T = 2
mu = 0.0
sigma = 0.5
S0 = 10
dt = 0.01
count_experiment = 1000

procent = [0, 0.15, 0.20, 0.25, 0.30]
amounts_S = [1000,1000,2000,4000,8000]
r_fin = 10
procent_loss = 2
r = 10
count_step = [0]*(len(amounts_S) + 1)
size_profit = [0]*(len(amounts_S) + 1)
count_days = [0]*(len(amounts_S) + 1)

profits_generated = np.zeros(count_experiment)
total_profit = 0

file_result_name = 'Result_Coins_2020_02_17_Art.csv'

print("Процент самого глубокого снижения (от стартовой цены): \n", prod(4, procent)*100, "%\n")

for i in range(0, count_experiment):
    gbm_t, gbm_t  = GBM_generator(mu, sigma, T, dt, S0)
    d = {'Date': gbm_t, 'Price': gbm_t}
    df = pd.DataFrame(data=d)
    transform_csv(df)
    df, profit = model_5k(df)
    profits_generated[i] = profit
    total_profit = total_profit + profit
    print(total_profit)
    # print(np.mean(profits_generated))
    # plt.plot(gbm_t, gbm_t, linewidth=0.1)


duration = timer() - start
print("Время расчетов в секундах: ", duration)

# print("Result TAB \n", df.head(), "\n")

df.to_csv(file_result_name, sep = ';', index=False,)
print("Файл создан: \n", file_result_name, "\n")


# Result = df[['Date','total_profit']]
# print(Result, profit)
# print(profits_generated)
print(total_profit)


# Generate data on commute times.
# size, scale = 1000, 10
commutes = pd.Series(profits_generated)

commutes.plot.hist(grid=True, bins=50, rwidth=0.9,
                   color='#607c8e')
plt.title('Commute Times Commuters')
plt.xlabel('Counts')
plt.ylabel('Commute Time')
plt.grid(axis='y', alpha=0.75)
plt.show()

Процент самого глубокого снижения (от стартовой цены): 
 35.699999999999996 %

1418.6731997534907
-1809.703019907617
-3934.675843187263
-4869.723871480107
-3028.248694142227
658.1192352759044
-814.0885523452889
-3066.073820115468
2693.7011133950846
5253.999165790807
7983.700794207374
10477.04862562545
11704.246732356532
14061.703172560798
15522.90120578841
14797.80985299715
16351.446161415868
17854.756298177836
20602.72177495738
15293.780021106075
12451.286345605338
10850.078949021245
12532.996297947046
10731.67601571968
13969.172609303461
11417.497029106584
9909.795759488336
7991.323770339946
6133.618956134682
4941.019064204106
5703.558221450039
4599.327020782512
3814.6416976643136
2766.16471150137
4928.3555844016455
7575.501552650712
5075.613462975016
2983.0518750208475
5221.978822453347
5065.5499164468165
9021.272454512899
13276.907943097613
12407.983802566463
10760.667124225116
14152.24486104336
15880.36392533827
17802.80499379272
19941.527803820918
17845.316374613318
15414.5034376

135621.30793487083
136823.31172272068
139860.28531374727
138490.3918247379
139722.52730199162
142627.83680743113
140931.7186803741
141359.85279495857
143744.57962910732
146453.33206513774
148801.157877301
147762.03242732296
148825.81659852163
147673.0303683744
150444.46599936252
152362.02670748412
154882.39588961896
153362.5392426512
155138.37235930245
154371.58126141192
153300.01233735663
154879.1367755969
153188.00330988358
150791.25062485907
149330.18731653836
150939.76418221116
149019.31330938992
147097.96073856013
148707.31490752383
147598.9131180147
146127.68560140464
148275.73576194505
151469.23845347832
153399.4149253087
157438.39802243715
159737.1341117034
159370.03823618044
161315.2452662723
163363.88342154166
165732.69084794712
164899.69607600936
163349.52967749976
166017.32681001636
167730.45375336776
166987.03746980158
168113.29409814833
169809.61886080704
164929.09257530642
162914.64107097077
164487.6972066779
163153.7568460746
161601.9327101656
161954.76255187002
156731.

184976.8288100154
185198.63413778518
186983.02728512295
190336.73968291076
188912.0516226188
181625.066512216
178499.1071742789
181288.11247268802
183366.42874341086
181745.18943366915
183146.75887349484
180198.56894808606
179816.71763396898
183438.02836987152
180904.21128653377
179066.61494401406
181046.30774355048
178150.88913773437
180253.47580843593
181852.77676569234
184011.4799325639
181586.32805060878
184362.24420726465
187785.63937336556
177961.3812844201
172890.58844557812
174071.90146378824
175749.33680870276
178849.42279900375
177905.35909726637
173153.43644294667
175323.22652692779
180361.5194680193
178534.50332104243
181011.2597438886
182249.32253630934
179841.1214993307
185868.81760743944
180699.75883045036
182779.4722654614
186826.3846826945
185241.80559429902
184009.58640879253
186540.36551631885
185561.56652956983
187752.40387265946
188988.91112085112
191242.23995091888
193666.22799703147
188160.48278651404
186987.6431634203
182658.75281903366
184212.89133677018
186152

<Figure size 640x480 with 1 Axes>